# Train CNN Models

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import tensorflow as tf

from model.dataset import SpectraDataset, DataGenerator, DataGeneratorRNN, split2feat

2022-12-28 16:37:00.839279: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


## Prepare Dataset

In [2]:
data_dir = 'data'
dataset = SpectraDataset(data_dir)

Reading train data
No. of materials : 22
[['Acrylic' '6']
 ['ENR' '6']
 ['EPDM' '6']
 ['HDPE' '6']
 ['LDPE' '6']
 ['Nylon' '6']
 ['PBAT' '6']
 ['PBS' '6']
 ['PC' '6']
 ['PEEK' '6']
 ['PEI' '6']
 ['PET' '6']
 ['PLA' '6']
 ['PMMA' '6']
 ['POM' '6']
 ['PP' '6']
 ['PS' '6']
 ['PTEE' '6']
 ['PU' '6']
 ['PVA' '6']
 ['PVC' '6']
 ['cellulose' '6']]
Read 132 spectra for training.
No. of data points/shifts per spectra (input shape) : (1802,)
No. of classes : 22
Reading test data
Read 88 spectra for testing.


In [3]:
dataset.get_binary_targets().shape

(132, 22)

In [4]:
dataset.get_training_data().shape

(132, 1802)

In [5]:
datagen = DataGenerator(dataset)

In [6]:
n_features = 24
datagen_rnn = DataGeneratorRNN(dataset, n_features=n_features)

In [7]:
datagen_rnn.data_shape

(132, 76, 24)

## Train

In [8]:
input_shape = dataset.get_input_shape()
num_class = dataset.get_num_class()

input_shape_rnn = datagen_rnn.data_shape

### ANN

In [9]:
from model.nn import ANN

ann = ANN(input_shape, num_class)
ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy', tf.keras.metrics.Recall()])
ann.summary()


ann.fit(datagen, epochs=20, validation_data=dataset.get_test_data())
# ann.fit(x=dataset.get_training_data(), 
#         y=dataset.get_binary_targets(), 
#         epochs=50, 
#         batch_size=32) #optimal

2022-12-28 16:37:03.617054: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-28 16:37:03.617651: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-28 16:37:03.651242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 16:37:03.651698: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce RTX 2070 computeCapability: 7.5
coreClock: 1.725GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-12-28 16:37:03.651713: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-12-28 16:37:03.652710: I tensorflow/stream_executor/platform/de

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1802)]            0         
_________________________________________________________________
dense (Dense)                (None, 200)               360600    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 22)                4422      
Total params: 405,222
Trainable params: 405,222
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1/4 [======>.......................] - ETA: 1s - loss: 0.6902 - binary_accuracy: 0.5256 - recall: 0.5625

2022-12-28 16:37:04.717182: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


4/4 [==============================] - 1s 123ms/step - loss: 0.5043 - binary_accuracy: 0.7536 - recall: 0.2625 - val_loss: 0.2104 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 2/20
4/4 [==============================] - 0s 15ms/step - loss: 0.1983 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1837 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 3/20
4/4 [==============================] - 0s 18ms/step - loss: 0.1842 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1845 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 4/20
4/4 [==============================] - 0s 17ms/step - loss: 0.1849 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1841 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 5/20
4/4 [==============================] - 0s 18ms/step - loss: 0.1820 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1812 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 6/20
4/4 [===

### AlexNet

In [10]:
# from model.nn import AlexNet

# alexnet = AlexNet(input_shape, num_class)
# alexnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# alexnet.summary()

# alexnet.fit(datagen, epochs=200)

### VGG16

In [11]:
# from model.nn import VGG16

# vgg16 = VGG16(input_shape, num_class)
# vgg16.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# vgg16.summary()

# vgg16.fit(datagen, epochs=200, validation_data=dataset.get_test_data())

### ResNet50

In [12]:
# from model.nn import ResNet50

# resnet = ResNet50(input_shape, num_class)
# resnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# resnet.summary()

# resnet.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

### DenseNet

In [13]:
# from model.nn import DenseNet

# densenet = DenseNet(input_shape, num_class)
# densenet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# densenet.summary()

# densenet.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

### LSTM

In [14]:
X_test, y_test = dataset.get_test_data()
X_test = split2feat(X_test, n_features)

In [15]:
from model.rnn import VanLSTM

lstm = VanLSTM(input_shape_rnn, num_class)
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
lstm.summary()

lstm.fit(datagen_rnn, epochs=20, validation_data=(X_test,y_test))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 76, 24)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 200)               180000    
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 22)                4422      
Total params: 264,822
Trainable params: 264,822
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


2022-12-28 16:37:08.001587: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


4/4 [==============================] - 2s 109ms/step - loss: 0.6892 - binary_accuracy: 0.7224 - val_loss: 0.6783 - val_binary_accuracy: 0.9545
Epoch 2/20
4/4 [==============================] - 0s 19ms/step - loss: 0.6318 - binary_accuracy: 0.9545 - val_loss: 0.3281 - val_binary_accuracy: 0.9545
Epoch 3/20
4/4 [==============================] - 0s 19ms/step - loss: 0.2692 - binary_accuracy: 0.9545 - val_loss: 0.2171 - val_binary_accuracy: 0.9545
Epoch 4/20
4/4 [==============================] - 0s 20ms/step - loss: 0.2199 - binary_accuracy: 0.9545 - val_loss: 0.2093 - val_binary_accuracy: 0.9545
Epoch 5/20
4/4 [==============================] - 0s 18ms/step - loss: 0.2033 - binary_accuracy: 0.9545 - val_loss: 0.1906 - val_binary_accuracy: 0.9545
Epoch 6/20
4/4 [==============================] - 0s 20ms/step - loss: 0.1910 - binary_accuracy: 0.9545 - val_loss: 0.1936 - val_binary_accuracy: 0.9545
Epoch 7/20
4/4 [==============================] - 0s 14ms/step - loss: 0.1921 - binary_accur

## Test

In [16]:
from utils import multilabel_classification_report

In [17]:
multilabel_classification_report(dataset, ann)

3/3 [==============================] - 0s 2ms/step - loss: 0.1360 - binary_accuracy: 0.9613 - recall: 0.1477
            precision     recall   f1-score    support
Acrylic          0.00       0.00       0.00          4
ENR              1.00       1.00       1.00          4
EPDM             0.00       0.00       0.00          4
HDPE             0.00       0.00       0.00          4
LDPE             0.00       0.00       0.00          4
Nylon            1.00       0.75       0.86          4
PBAT             0.00       0.00       0.00          4
PBS              0.00       0.00       0.00          4
PC               0.00       0.00       0.00          4
PEEK             0.00       0.00       0.00          4
PEI              0.00       0.00       0.00          4
PET              0.00       0.00       0.00          4
PLA              1.00       0.75       0.86          4
PMMA             0.00       0.00       0.00          4
POM              0.00       0.00       0.00          4
PP         

/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/an

In [18]:
# multilabel_classification_report(dataset, alexnet)

In [19]:
# multilabel_classification_report(dataset, vgg16)

In [20]:
# multilabel_classification_report(dataset, resnet)

In [21]:
# multilabel_classification_report(dataset, densenet)

In [22]:
multilabel_classification_report(dataset, lstm, n_features)

3/3 [==============================] - 0s 3ms/step - loss: 0.1852 - binary_accuracy: 0.9545
            precision     recall   f1-score    support
Acrylic          0.00       0.00       0.00          4
ENR              0.00       0.00       0.00          4
EPDM             0.00       0.00       0.00          4
HDPE             0.00       0.00       0.00          4
LDPE             0.00       0.00       0.00          4
Nylon            0.00       0.00       0.00          4
PBAT             0.00       0.00       0.00          4
PBS              0.00       0.00       0.00          4
PC               0.00       0.00       0.00          4
PEEK             0.00       0.00       0.00          4
PEI              0.00       0.00       0.00          4
PET              0.00       0.00       0.00          4
PLA              0.00       0.00       0.00          4
PMMA             0.00       0.00       0.00          4
POM              0.00       0.00       0.00          4
PP               0.00       

/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/anaconda3/envs/wd/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoyo/an